In [3]:
#Creation of pipeline

Modules : Moviepy

In [2]:
#conversion from vid to wav : 

In [ ]:
from moviepy.editor import VideoFileClip

# Path to the input video file
input_video_path = r'D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\Video\test2.mp4'


# Path to the output audio file
output_audio_path = r'D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\Audio_Data\audio2.wav'

# Load the video clip
video_clip = VideoFileClip(input_video_path)

# Extract audio from the video clip
audio_clip = video_clip.audio

# Write the audio to a WAV file
audio_clip.write_audiofile(output_audio_path)

# Close the video clip
video_clip.close()


In [ ]:
#Using wav2vec2 for audio transcribing : loading wav , training , saving
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import librosa

# Load pre-trained Wav2Vec2 model and processor
wav2vec2_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
wav2vec2_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# Load audio file
audio_file_path = r'D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\NPTEL\Audio\01.wav'

# Transcribe audio in batches using Wav2Vec2
def transcribe_audio_batch(audio_file_path, batch_size=10, chunk_size=5, interval=10):
    audio_input, _ = librosa.load(audio_file_path, sr=16000)

    # Split audio into chunks
    chunks = [audio_input[i:i + chunk_size * 16000] for i in range(0, len(audio_input), chunk_size * 16000)]

    transcriptions = []

    # Process chunks in batches
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i + batch_size]

        input_values = wav2vec2_processor(batch, sampling_rate=16000, return_tensors="pt", padding=True).input_values
        logits = wav2vec2_model(input_values).logits

        predicted_ids = torch.argmax(logits, dim=-1)
        batch_transcriptions = wav2vec2_processor.batch_decode(predicted_ids)

        transcriptions.extend(batch_transcriptions)

    return transcriptions

# Transcribe audio in batches and save to a .txt file
transcriptions = transcribe_audio_batch(audio_file_path)
print("Transcriptions:", transcriptions)

# Filter out empty strings from transcriptions
transcriptions = [transcription.strip() for transcription in transcriptions if transcription.strip()]

# Join transcriptions into a single string without spaces between words
transcriptions_text = "".join(transcriptions)

# Save transcriptions to a .txt file
output_file = "transcriptions2.txt"
with open(output_file, "w") as file:
    file.write(transcriptions_text)
y

In [4]:
#Optimizing the transcribed text , removing filler words 
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Load the transcribed text from the file
file_path = 'transcriptions2.txt'
with open(file_path, 'r') as file:
    text = file.read()

# Tokenize the text into words
words = word_tokenize(text)

# Define a set of stopwords
english_stopwords = set(stopwords.words('english'))

# Remove punctuation from string.punctuation
punctuation = set(string.punctuation)

# Combine all stopwords
all_stopwords = english_stopwords.union(punctuation)

# Remove stopwords and punctuation
filtered_words = [word for word in words if word.lower() not in all_stopwords]

# Join the filtered words back into a string
filtered_text = ' '.join(filtered_words)

# Save the filtered text to a new file
output_file_path = 'transcriptions2_test1.txt'
with open(output_file_path, 'w') as file:
    file.write(filtered_text)

print("Filtered text saved to:", output_file_path)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pavan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pavan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Filtered text saved to: transcriptions2_test1.txt


In [1]:
from spellchecker import SpellChecker

# Initialize the spellchecker
spell = SpellChecker()

# Open the file with filtered text without stopwords
filtered_text_file_path = 'transcriptions2.txt'
with open(filtered_text_file_path, 'r') as file:
    filtered_text = file.read()

# Tokenize the filtered text into words
filtered_words = filtered_text.split()

# Correct each word using spellchecker
corrected_words = []
for word in filtered_words:
    corrected_word = spell.correction(word)
    if corrected_word is not None:
        corrected_words.append(corrected_word)

# Join the corrected words back into a string
corrected_text = ' '.join(corrected_words)

# Save the corrected text to a new file
output_file_path = 'transcriptions2_corrected_test1.txt'
with open(output_file_path, 'w') as file:
    file.write(corrected_text)

print("Corrected text saved to:", output_file_path)


Corrected text saved to: transcriptions2_corrected_test1.txt


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Read the transcribed audio text from the file
transcribed_audio_file_path = 'transcriptions2_nostop_corrected.txt'
with open(transcribed_audio_file_path, 'r') as file:
    transcribed_audio_text = file.read()

# Given input text
input_text = "human"

# Preprocess the text: tokenize and vectorize
vectorizer = CountVectorizer().fit([transcribed_audio_text, input_text])
vectorized_text = vectorizer.transform([transcribed_audio_text, input_text])

# Compute the cosine similarity between the two texts
cosine_sim = cosine_similarity(vectorized_text)[0][1]

# Print the cosine similarity score
print("Cosine Similarity Score:", cosine_sim)


Cosine Similarity Score: 0.23635285260380087


In [4]:
from nltk.metrics.distance import jaro_winkler_similarity

# Calculate Jaro-Winkler similarity
jaro_winkler_sim = jaro_winkler_similarity(transcribed_audio_text, input_text)

# Print Jaro-Winkler similarity score
print("Jaro-Winkler Similarity Score:", jaro_winkler_sim)


Jaro-Winkler Similarity Score: 0.5334493560737905


In [19]:
# import nltk
# from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
# import string
# from textblob import TextBlob
# import enchant

# # Download NLTK resources
# nltk.download('punkt')
# nltk.download('stopwords')

# # Load the transcribed text from the file
# file_path = 'transcriptions2.txt'
# with open(file_path, 'r') as file:
#     text = file.read()

# # Phase 1: Tokenization
# words = word_tokenize(text)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pavan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pavan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
# # Phase 2: Spell Checking and Correction using TextBlob
# corrected_words = [str(TextBlob(word).correct()) for word in words]


In [5]:
# # Phase 3: Stopword Removal and Punctuation using NLTK
# english_stopwords = set(stopwords.words('english'))
# punctuation = set(string.punctuation)
# filtered_words = [word for word in corrected_words if word.lower() not in english_stopwords.union(punctuation)]


In [13]:
# from langdetect import detect

# # Phase 4: Remove non-English words using langdetect
# filtered_words_en = []

# for word in filtered_words:
#     try:
#         if detect(word) == 'en':
#             filtered_words_en.append(word)
#     except:
#         # Handle the case when language detection fails
#         pass

# filtered_words = filtered_words_en


In [21]:
# # Join the filtered words back into a string
# filtered_text = ' '.join(corrected_words)

# # Save the filtered text to a new file
# output_file_path = 'preprocessed_text_3.txt'
# with open(output_file_path, 'w') as file:
#     file.write(filtered_text)

# print("Preprocessed text saved to:", output_file_path)


Preprocessed text saved to: preprocessed_text_3.txt


In [15]:
# # Loading the preprocessed text file - 1
# file_path = 'transcriptions2_corrected_test1.txt'
# with open(file_path, 'r') as file:
#     text = file.read()

# # Split the text into words
# words = text.split()

# # Calculate the word count
# word_count = len(words)
# print("Word Count:", word_count)


Word Count: 3513


In [16]:
# # Calculate the unique word count
# unique_words = set(words)
# unique_word_count = len(unique_words)
# print("Unique Word Count:", unique_word_count)


Unique Word Count: 1182


In [ ]:
#Failed. modifying to find out which phase is causing failure

In [22]:
# # Loading the preprocessed text file -2 
# file_path = 'preprocessed_text_3.txt'
# with open(file_path, 'r') as file:
#     text = file.read()

# # Split the text into words
# words = text.split()

# # Calculate the word count
# word_count = len(words)
# print("Word Count:", word_count)


Word Count: 3724


In [18]:
# # Calculate the unique word count
# unique_words = set(words)
# unique_word_count = len(unique_words)
# print("Unique Word Count:", unique_word_count)


Unique Word Count: 87


In [10]:
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import librosa
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.metrics.distance import jaro_winkler_similarity
import string
from moviepy.editor import VideoFileClip
from spellchecker import SpellChecker
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Define file paths
#input_video_path = r'D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\Video\test2.mp4'
#output_audio_path = r'D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\Audio_Data\audio2.wav'
audio_file_path = r'D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\NPTEL\Audio\01.wav'
output_file = "transcriptions2.txt"
output_file_path = 'transcriptions2_test1.txt'
filtered_text_file_path = 'transcriptions2.txt'
output_file_path_corrected = 'transcriptions2_corrected_test1.txt'
transcribed_audio_file_path = 'transcriptions2_nostop_corrected.txt'


def convert_video_to_audio(input_video_path, output_audio_path):
    """Converts a video file to audio."""
    video_clip = VideoFileClip(input_video_path)
    audio_clip = video_clip.audio
    audio_clip.write_audiofile(output_audio_path)
    video_clip.close()


def transcribe_audio_batch(audio_file_path, batch_size=10, chunk_size=5, interval=10):
    """Transcribes audio in batches using Wav2Vec2."""
    wav2vec2_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
    wav2vec2_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

    audio_input, _ = librosa.load(audio_file_path, sr=16000)
    chunks = [audio_input[i:i + chunk_size * 16000] for i in range(0, len(audio_input), chunk_size * 16000)]

    transcriptions = []

    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i + batch_size]
        input_values = wav2vec2_processor(batch, sampling_rate=16000, return_tensors="pt", padding=True).input_values
        logits = wav2vec2_model(input_values).logits
        predicted_ids = torch.argmax(logits, dim=-1)
        batch_transcriptions = wav2vec2_processor.batch_decode(predicted_ids)
        transcriptions.extend(batch_transcriptions)

    return transcriptions


def optimize_text(file_path, output_file_path):
    """Optimizes transcribed text by removing stopwords and punctuation."""
    with open(file_path, 'r') as file:
        text = file.read()

    words = word_tokenize(text)
    english_stopwords = set(stopwords.words('english'))
    punctuation = set(string.punctuation)
    all_stopwords = english_stopwords.union(punctuation)

    filtered_words = [word for word in words if word.lower() not in all_stopwords]
    filtered_text = ' '.join(filtered_words)

    with open(output_file_path, 'w') as file:
        file.write(filtered_text)


def correct_spelling(filtered_text_file_path, output_file_path_corrected):
    """Corrects spelling errors in the filtered text."""
    spell = SpellChecker()

    with open(filtered_text_file_path, 'r') as file:
        filtered_text = file.read()

    filtered_words = filtered_text.split()
    corrected_words = [spell.correction(word) for word in filtered_words if spell.correction(word) is not None]
    corrected_text = ' '.join(corrected_words)

    with open(output_file_path_corrected, 'w') as file:
        file.write(corrected_text)


def calculate_cosine_similarity(transcribed_audio_file_path, input_text):
    """Calculates cosine similarity between transcribed audio text and input text."""
    with open(transcribed_audio_file_path, 'r') as file:
        transcribed_audio_text = file.read()

    vectorizer = CountVectorizer().fit([transcribed_audio_text, input_text])
    vectorized_text = vectorizer.transform([transcribed_audio_text, input_text])
    cosine_sim = cosine_similarity(vectorized_text)[0][1]

    return cosine_sim


def calculate_jaro_winkler_similarity(transcribed_audio_text, input_text):
    """Calculates Jaro-Winkler similarity between transcribed audio text and input text."""
    return jaro_winkler_similarity(transcribed_audio_text, input_text)


# Convert video to audio
convert_video_to_audio(input_video_path, output_audio_path)

# Transcribe audio
transcriptions = transcribe_audio_batch(audio_file_path)
transcriptions_text = "".join(transcriptions)
with open(output_file, "w") as file:
    file.write(transcriptions_text)

# Optimize transcribed text
optimize_text(output_file, output_file_path)

# Correct spelling errors
correct_spelling(output_file_path, output_file_path_corrected)

# Calculate cosine similarity
input_text = "java"
cosine_sim = calculate_cosine_similarity(transcribed_audio_file_path, input_text)
print("Cosine Similarity Score:", cosine_sim)

# Calculate Jaro-Winkler similarity
jaro_winkler_sim = calculate_jaro_winkler_similarity(transcribed_audio_text, input_text)
print("Jaro-Winkler Similarity Score:", jaro_winkler_sim)


MoviePy - Writing audio in D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\Audio_Data\audio2.wav


MoviePy - Done.


Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

Cosine Similarity Score: 0.0
Jaro-Winkler Similarity Score: 0.583426151525699


In [33]:
input_text = "AI"
cosine_sim = calculate_cosine_similarity(transcribed_audio_file_path, input_text)
print("Cosine Similarity Score:", cosine_sim)

Cosine Similarity Score: 0.04924017762579185


In [30]:
input_text = "you"

# Calculate Jaro-Winkler similarity
jaro_winkler_sim = calculate_jaro_winkler_similarity(transcribed_audio_text, input_text)
print("Jaro-Winkler Similarity Score:", jaro_winkler_sim)

Jaro-Winkler Similarity Score: 0.5556251691998298


In [34]:
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import librosa
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.metrics.distance import jaro_winkler_similarity
import string
from moviepy.editor import VideoFileClip
from spellchecker import SpellChecker
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import concurrent.futures

# Define file paths
audio_file_path = r'D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\NPTEL\Audio\01.wav'
output_file = "transcriptions2.txt"
output_file_path = 'transcriptions2_test1.txt'
filtered_text_file_path = 'transcriptions2.txt'
output_file_path_corrected = 'transcriptions2_corrected_test1.txt'
transcribed_audio_file_path = 'transcriptions2_nostop_corrected.txt'

# Define video and audio conversion paths
input_video_path = r'D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\Video\test2.mp4'
output_audio_path = r'D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\Audio_Data\audio2.wav'


def convert_video_to_audio(input_video_path, output_audio_path):
    """Converts a video file to audio."""
    video_clip = VideoFileClip(input_video_path)
    audio_clip = video_clip.audio
    audio_clip.write_audiofile(output_audio_path)
    video_clip.close()


def transcribe_chunk(chunk, wav2vec2_processor, wav2vec2_model):
    """Transcribe a chunk of audio."""
    input_values = wav2vec2_processor(chunk, sampling_rate=16000, return_tensors="pt", padding=True).input_values
    logits = wav2vec2_model(input_values).logits
    transcription = wav2vec2_processor.batch_decode(torch.argmax(logits, dim=-1))
    return transcription


def transcribe_audio_parallel(audio_file_path, batch_size=10, chunk_size=5):
    """Transcribe audio in parallel using Wav2Vec2."""
    wav2vec2_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
    wav2vec2_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

    audio_input, _ = librosa.load(audio_file_path, sr=16000)
    chunks = [audio_input[i:i + chunk_size * 16000] for i in range(0, len(audio_input), chunk_size * 16000)]

    transcriptions = []

    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_chunk = {executor.submit(transcribe_chunk, chunk, wav2vec2_processor, wav2vec2_model): chunk
                           for chunk in chunks}

        for future in concurrent.futures.as_completed(future_to_chunk):
            try:
                transcription = future.result()
                transcriptions.extend(transcription)
            except Exception as e:
                print(f"Error processing chunk: {e}")

    return transcriptions


def optimize_text(file_path, output_file_path):
    """Optimizes transcribed text by removing stopwords and punctuation."""
    with open(file_path, 'r') as file:
        text = file.read()

    words = word_tokenize(text)
    english_stopwords = set(stopwords.words('english'))
    punctuation = set(string.punctuation)
    all_stopwords = english_stopwords.union(punctuation)

    filtered_words = [word for word in words if word.lower() not in all_stopwords]
    filtered_text = ' '.join(filtered_words)

    with open(output_file_path, 'w') as file:
        file.write(filtered_text)


def correct_spelling(filtered_text_file_path, output_file_path_corrected):
    """Corrects spelling errors in the filtered text."""
    spell = SpellChecker()

    with open(filtered_text_file_path, 'r') as file:
        filtered_text = file.read()

    filtered_words = filtered_text.split()
    corrected_words = [spell.correction(word) for word in filtered_words if spell.correction(word) is not None]
    corrected_text = ' '.join(corrected_words)

    with open(output_file_path_corrected, 'w') as file:
        file.write(corrected_text)


def calculate_cosine_similarity(transcribed_audio_file_path, input_text):
    """Calculates cosine similarity between transcribed audio text and input text."""
    with open(transcribed_audio_file_path, 'r') as file:
        transcribed_audio_text = file.read()

    vectorizer = CountVectorizer().fit([transcribed_audio_text, input_text])
    vectorized_text = vectorizer.transform([transcribed_audio_text, input_text])
    cosine_sim = cosine_similarity(vectorized_text)[0][1]

    return cosine_sim


def calculate_jaro_winkler_similarity(transcribed_audio_text, input_text):
    """Calculates Jaro-Winkler similarity between transcribed audio text and input text."""
    return jaro_winkler_similarity(transcribed_audio_text, input_text)


# Convert video to audio
convert_video_to_audio(input_video_path, output_audio_path)

# Transcribe audio in parallel
transcriptions = transcribe_audio_parallel(audio_file_path)

# Write transcriptions to file
transcriptions_text = "".join(transcriptions)
with open(output_file, "w") as file:
    file.write(transcriptions_text)

# Optimize transcribed text
optimize_text(output_file, output_file_path)

# Correct spelling errors
correct_spelling(output_file_path, output_file_path_corrected)

# Calculate cosine similarity
input_text = "java"
cosine_sim = calculate_cosine_similarity(transcribed_audio_file_path, input_text)
print("Cosine Similarity Score:", cosine_sim)

# Calculate Jaro-Winkler similarity
jaro_winkler_sim = calculate_jaro_winkler_similarity(transcribed_audio_text, input_text)
print("Jaro-Winkler Similarity Score:", jaro_winkler_sim)


MoviePy - Writing audio in D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\Audio_Data\audio2.wav


MoviePy - Done.


Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

Cosine Similarity Score: 0.0
Jaro-Winkler Similarity Score: 0.583426151525699


In [1]:
import os
import concurrent.futures
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import librosa
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.metrics.distance import jaro_winkler_similarity
import string
from moviepy.editor import VideoFileClip
from spellchecker import SpellChecker
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Define file paths
audio_directory = r'D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\NPTEL\Audio'
output_directory = r'D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\Output'

def convert_video_to_audio(input_video_path, output_audio_path):
    """Converts a video file to audio."""
    video_clip = VideoFileClip(input_video_path)
    audio_clip = video_clip.audio
    audio_clip.write_audiofile(output_audio_path)
    video_clip.close()

def transcribe_chunk(chunk, wav2vec2_processor, wav2vec2_model):
    """Transcribe a chunk of audio."""
    input_values = wav2vec2_processor(chunk, sampling_rate=16000, return_tensors="pt", padding=True).input_values
    logits = wav2vec2_model(input_values).logits
    transcription = wav2vec2_processor.batch_decode(torch.argmax(logits, dim=-1))
    return transcription

def transcribe_audio_parallel(audio_file_path, wav2vec2_processor, wav2vec2_model, chunk_size=5):
    """Transcribe audio in parallel using Wav2Vec2."""
    audio_input, _ = librosa.load(audio_file_path, sr=16000)
    chunks = [audio_input[i:i + chunk_size * 16000] for i in range(0, len(audio_input), chunk_size * 16000)]
    transcriptions = []

    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_chunk = {executor.submit(transcribe_chunk, chunk, wav2vec2_processor, wav2vec2_model): chunk
                           for chunk in chunks}

        for future in concurrent.futures.as_completed(future_to_chunk):
            try:
                transcription = future.result()
                transcriptions.extend(transcription)
            except Exception as e:
                print(f"Error processing chunk: {e}")

    return transcriptions

def optimize_text(text):
    """Optimizes transcribed text by removing stopwords and punctuation."""
    words = word_tokenize(text)
    english_stopwords = set(stopwords.words('english'))
    punctuation = set(string.punctuation)
    all_stopwords = english_stopwords.union(punctuation)
    filtered_words = [word for word in words if word.lower() not in all_stopwords]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

def correct_spelling(text):
    """Corrects spelling errors in the text."""
    spell = SpellChecker()
    filtered_words = text.split()
    corrected_words = [spell.correction(word) for word in filtered_words if spell.correction(word) is not None]
    corrected_text = ' '.join(corrected_words)
    return corrected_text

def calculate_cosine_similarity(transcribed_audio_text, input_text):
    """Calculates cosine similarity between transcribed audio text and input text."""
    vectorizer = CountVectorizer().fit([transcribed_audio_text, input_text])
    vectorized_text = vectorizer.transform([transcribed_audio_text, input_text])
    cosine_sim = cosine_similarity(vectorized_text)[0][1]
    return cosine_sim

def calculate_jaro_winkler_similarity(transcribed_audio_text, input_text):
    """Calculates Jaro-Winkler similarity between transcribed audio text and input text."""
    return jaro_winkler_similarity(transcribed_audio_text, input_text)

if __name__ == "__main__":
    # Load pre-trained Wav2Vec2 model and processor
    wav2vec2_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
    wav2vec2_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

    # Convert video to audio
 #   input_video_path = r'D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\Video\test2.mp4'
 #   output_audio_path = r'D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\Audio_Data\audio2.wav'
 #   convert_video_to_audio(input_video_path, output_audio_path)

    # Transcribe and process each audio file in parallel
    for audio_file in os.listdir(audio_directory):
        if audio_file.endswith('.wav'):
            audio_file_path = os.path.join(audio_directory, audio_file)
            transcriptions = transcribe_audio_parallel(audio_file_path, wav2vec2_processor, wav2vec2_model)
            transcriptions_text = "".join(transcriptions)
            output_file = os.path.join(output_directory, f"{os.path.splitext(audio_file)[0]}_transcription.txt")
            with open(output_file, "w") as file:
                file.write(transcriptions_text)
            optimized_text = optimize_text(transcriptions_text)
            corrected_text = correct_spelling(optimized_text)
            with open(output_file, "w") as file:
                file.write(corrected_text)
            input_text = "state space search"  # Example input text for similarity calculation
            cosine_sim = calculate_cosine_similarity(corrected_text, input_text)
            jaro_winkler_sim = calculate_jaro_winkler_similarity(corrected_text, input_text)
            print(f"File: {audio_file}, Cosine Similarity: {cosine_sim}, Jaro-Winkler Similarity: {jaro_winkler_sim}")


Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

KeyboardInterrupt: 

In [21]:
import os
import concurrent.futures
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import librosa
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.metrics.distance import jaro_winkler_similarity
import string
from moviepy.editor import VideoFileClip
from spellchecker import SpellChecker
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Define file paths
audio_directory = r'D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\NPTEL\Audio'
output_directory = r'D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\Output'

def convert_video_to_audio(input_video_path, output_audio_path):
    """Converts a video file to audio."""
    video_clip = VideoFileClip(input_video_path)
    audio_clip = video_clip.audio
    audio_clip.write_audiofile(output_audio_path)
    video_clip.close()

def transcribe_chunk(chunk, wav2vec2_processor, wav2vec2_model):
    """Transcribe a chunk of audio."""
    input_values = wav2vec2_processor(chunk, sampling_rate=16000, return_tensors="pt", padding=True).input_values
    logits = wav2vec2_model(input_values).logits
    transcription = wav2vec2_processor.batch_decode(torch.argmax(logits, dim=-1))
    return transcription

def transcribe_audio_parallel(audio_file_path, wav2vec2_processor, wav2vec2_model, chunk_size=5):
    """Transcribe audio in parallel using Wav2Vec2."""
    audio_input, _ = librosa.load(audio_file_path, sr=16000)
    chunks = [audio_input[i:i + chunk_size * 16000] for i in range(0, len(audio_input), chunk_size * 16000)]
    transcriptions = []

    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_chunk = {executor.submit(transcribe_chunk, chunk, wav2vec2_processor, wav2vec2_model): chunk
                           for chunk in chunks}

        for future in concurrent.futures.as_completed(future_to_chunk):
            try:
                transcription = future.result()
                transcriptions.extend(transcription)
            except Exception as e:
                print(f"Error processing chunk: {e}")

    return transcriptions

def optimize_text(text):
    """Optimizes transcribed text by removing stopwords and punctuation."""
    words = word_tokenize(text)
    english_stopwords = set(stopwords.words('english'))
    punctuation = set(string.punctuation)
    all_stopwords = english_stopwords.union(punctuation)
    filtered_words = [word for word in words if word.lower() not in all_stopwords]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

def correct_spelling(text):
    """Corrects spelling errors in the text."""
    spell = SpellChecker()
    filtered_words = text.split()
    corrected_words = [spell.correction(word) for word in filtered_words if spell.correction(word) is not None]
    corrected_text = ' '.join(corrected_words)
    return corrected_text

def calculate_cosine_similarity(transcribed_audio_text, input_text):
    """Calculates cosine similarity between transcribed audio text and input text."""
    vectorizer = CountVectorizer().fit([transcribed_audio_text, input_text])
    vectorized_text = vectorizer.transform([transcribed_audio_text, input_text])
    cosine_sim = cosine_similarity(vectorized_text)[0][1]
    return cosine_sim

def calculate_jaro_winkler_similarity(transcribed_audio_text, input_text):
    """Calculates Jaro-Winkler similarity between transcribed audio text and input text."""
    return jaro_winkler_similarity(transcribed_audio_text, input_text)

if __name__ == "__main__":
    # Load pre-trained Wav2Vec2 model and processor
    wav2vec2_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
    wav2vec2_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
    input_text = "state space search"  # Example input text for similarity calculation

    # Convert video to audio
 #   input_video_path = r'D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\Video\test2.mp4'
 #   output_audio_path = r'D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\Audio_Data\audio2.wav'
 #   convert_video_to_audio(input_video_path, output_audio_path)

    # Transcribe and process each audio file in parallel
    for audio_file in os.listdir(audio_directory):
        if audio_file.endswith('.wav'):
            audio_file_path = os.path.join(audio_directory, audio_file)
            output_file = os.path.join(output_directory, f"{os.path.splitext(audio_file)[0]}_transcription.txt")

            # Check if transcription file already exists
            if not os.path.exists(output_file):
                transcriptions = transcribe_audio_parallel(audio_file_path, wav2vec2_processor, wav2vec2_model)
                transcriptions_text = "".join(transcriptions)
                with open(output_file, "w") as file:
                    file.write(transcriptions_text)
                optimized_text = optimize_text(transcriptions_text)
                corrected_text = correct_spelling(optimized_text)
                with open(output_file, "w") as file:
                    file.write(corrected_text)
                cosine_sim = calculate_cosine_similarity(corrected_text, input_text)
                jaro_winkler_sim = calculate_jaro_winkler_similarity(corrected_text, input_text)
                print(f"File: {audio_file}, Cosine Similarity: {cosine_sim}, Jaro-Winkler Similarity: {jaro_winkler_sim}")
            else:
                print(f"Transcription file already exists for {audio_file}. Skipping transcription.")

KeyboardInterrupt: 

In [ ]:
import os
import shutil
import concurrent.futures
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import librosa
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from spellchecker import SpellChecker
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string
from moviepy.editor import VideoFileClip

# Define the directory containing the videos
video_directory = r'D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\NPTEL\Video'

# Define the parent directory where the audio and transcriptions will be stored
parent_audio_directory = r'D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\NPTEL'

# Define the reference text for similarity calculation
input_text = "linear regression"  

# Load pre-trained Wav2Vec2 model and processor
wav2vec2_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
wav2vec2_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# Function to convert video to audio
def convert_video_to_audio(input_video_path, output_audio_path):
    """Converts a video file to audio."""
    video_clip = VideoFileClip(input_video_path)
    audio_clip = video_clip.audio
    audio_clip.write_audiofile(output_audio_path)
    video_clip.close()

# Function to transcribe audio chunks
def transcribe_chunk(chunk, wav2vec2_processor, wav2vec2_model):
    """Transcribe a chunk of audio."""
    input_values = wav2vec2_processor(chunk, sampling_rate=16000, return_tensors="pt", padding=True).input_values
    logits = wav2vec2_model(input_values).logits
    transcription = wav2vec2_processor.batch_decode(torch.argmax(logits, dim=-1))
    return transcription

# Function to transcribe audio in parallel
def transcribe_audio_parallel(audio_file_path, wav2vec2_processor, wav2vec2_model, chunk_size=5):
    """Transcribe audio in parallel using Wav2Vec2."""
    audio_input, _ = librosa.load(audio_file_path, sr=16000)
    chunks = [audio_input[i:i + chunk_size * 16000] for i in range(0, len(audio_input), chunk_size * 16000)]
    transcriptions = []

    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_chunk = {executor.submit(transcribe_chunk, chunk, wav2vec2_processor, wav2vec2_model): chunk
                           for chunk in chunks}

        for future in concurrent.futures.as_completed(future_to_chunk):
            try:
                transcription = future.result()
                transcriptions.extend(transcription)
            except Exception as e:
                print(f"Error processing chunk: {e}")

    return transcriptions

# Function to optimize transcribed text
def optimize_text(text):
    """Optimizes transcribed text by removing stopwords and punctuation."""
    words = word_tokenize(text)
    english_stopwords = set(stopwords.words('english'))
    punctuation = set(string.punctuation)
    all_stopwords = english_stopwords.union(punctuation)
    filtered_words = [word for word in words if word.lower() not in all_stopwords]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

# Function to correct spelling errors
def correct_spelling(text):
    """Corrects spelling errors in the text."""
    spell = SpellChecker()
    filtered_words = text.split()
    corrected_words = [spell.correction(word) for word in filtered_words if spell.correction(word) is not None]
    corrected_text = ' '.join(corrected_words)
    return corrected_text

# Function to calculate cosine similarity
def calculate_cosine_similarity(transcribed_audio_text, input_text):
    """Calculates cosine similarity between transcribed audio text and input text."""
    vectorizer = CountVectorizer().fit([transcribed_audio_text, input_text])
    vectorized_text = vectorizer.transform([transcribed_audio_text, input_text])
    cosine_sim = cosine_similarity(vectorized_text)[0][1]
    return cosine_sim

if __name__ == "__main__":
    # Convert video to audio
    for video_file in os.listdir(video_directory):
        if video_file.endswith('.mp4'):
            video_path = os.path.join(video_directory, video_file)
            audio_directory = os.path.join(parent_audio_directory, 'audio_KBVRS', os.path.splitext(video_file)[0])
            audio_path = os.path.join(audio_directory, os.path.splitext(video_file)[0] + '.wav')
            transcription_file = os.path.join(audio_directory, os.path.splitext(video_file)[0] + '_transcription.txt')

            # Check if transcription directory already exists
            if not os.path.exists(audio_directory):
                os.makedirs(audio_directory)

                # Convert video to audio
                convert_video_to_audio(video_path, audio_path)

    # Transcribe audio
    for video_file in os.listdir(video_directory):
        if video_file.endswith('.mp4'):
            video_path = os.path.join(video_directory, video_file)
            audio_directory = os.path.join(parent_audio_directory, 'audio_KBVRS', os.path.splitext(video_file)[0])
            audio_path = os.path.join(audio_directory, os.path.splitext(video_file)[0] + '.wav')
            transcription_file = os.path.join(audio_directory, os.path.splitext(video_file)[0] + '_transcription.txt')

            # Check if transcription file already exists
            if not os.path.exists(transcription_file):
                # Transcribe audio
                transcriptions = transcribe_audio_parallel(audio_path, wav2vec2_processor, wav2vec2_model)
                transcriptions_text = "".join(transcriptions)

                # Write transcribed text to file
                with open(transcription_file, "w") as file:
                    file.write(transcriptions_text)

                # Optimize text and correct spelling
                optimized_text = optimize_text(transcriptions_text)
                corrected_text = correct_spelling(optimized_text)

                # Calculate cosine similarity
                cosine_sim = calculate_cosine_similarity(corrected_text, input_text)

                print(f"Video: {video_file}, Cosine Similarity: {cosine_sim}")

                # Hide the directory
                if os.name == 'nt':  # Windows
                    ctypes.windll.kernel32.SetFileAttributesW(audio_directory, 2)  # 2 is the code for hidden attribute
                else:  # Unix-like
                    os.rename(audio_directory, os.path.join(os.path.dirname(audio_directory), '.' + os.path.basename(audio_directory)))
            else:
                print(f"Transcription file already exists for {video_file}. Skipping transcription.")


Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

MoviePy - Writing audio in D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\NPTEL\audio_KBVRS\02\02.wav


MoviePy - Done.
MoviePy - Writing audio in D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\NPTEL\audio_KBVRS\03\03.wav


MoviePy - Done.
MoviePy - Writing audio in D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\NPTEL\audio_KBVRS\04\04.wav


MoviePy - Done.
MoviePy - Writing audio in D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\NPTEL\audio_KBVRS\05\05.wav


...

MoviePy - Done.
MoviePy - Writing audio in D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\NPTEL\audio_KBVRS\07\07.wav


MoviePy - Done.
MoviePy - Writing audio in D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\NPTEL\audio_KBVRS\09\09.wav


MoviePy - Done.
MoviePy - Writing audio in D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\NPTEL\audio_KBVRS\11\11.wav


MoviePy - Done.
MoviePy - Writing audio in D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\NPTEL\audio_KBVRS\29\29.wav


MoviePy - Done.
Transcription file already exists for 01.mp4. Skipping transcription.
Video: 02.mp4, Cosine Similarity: 0.0
Video: 03.mp4, Cosine Similarity: 0.004349470810331771
Video: 04.mp4, Cosine Similarity: 0.0
Video: 05.mp4, Cosine Similarity: 0.0
Video: 06.mp4, Cosine Similarity: 0.0
Video: 07.mp4, Cosine Similarity: 0.0


In [10]:
import os
from nltk.metrics.distance import jaro_winkler_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Define the directory containing the transcribed text files
transcribed_directory = r'D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\Output'

# Reference text for similarity calculation
reference_text = "linear regression"

# Function to calculate cosine similarity
def calculate_cosine_similarity(transcribed_text, reference_text):
    vectorizer = CountVectorizer().fit([transcribed_text, reference_text])
    vectorized_text = vectorizer.transform([transcribed_text, reference_text])
    cosine_sim = cosine_similarity(vectorized_text)[0][1]
    return cosine_sim

# Function to calculate Jaro-Winkler similarity
def calculate_jaro_winkler_similarity(transcribed_text, reference_text):
    return jaro_winkler_similarity(transcribed_text, reference_text)

# Iterate through each text file in the directory
for filename in os.listdir(transcribed_directory):
    if filename.endswith('.txt'):
        file_path = os.path.join(transcribed_directory, filename)
        
        # Read the content of the text file
        with open(file_path, 'r') as file:
            transcribed_text = file.read()

        # Calculate cosine similarity
        cosine_sim = calculate_cosine_similarity(transcribed_text, reference_text)
        
        # Calculate Jaro-Winkler similarity
        jaro_winkler_sim = calculate_jaro_winkler_similarity(transcribed_text, reference_text)

        # Print or store the similarity scores
        print(f"File: {filename}, Cosine Similarity: {cosine_sim}, Jaro-Winkler Similarity: {jaro_winkler_sim}")


File: 01_transcription.txt, Cosine Similarity: 0.0, Jaro-Winkler Similarity: 0.5102029545005805
File: 02_transcription.txt, Cosine Similarity: 0.0, Jaro-Winkler Similarity: 0.5296761915218724
File: 03_transcription.txt, Cosine Similarity: 0.004343804386977241, Jaro-Winkler Similarity: 0.529726614504008
File: 04_transcription.txt, Cosine Similarity: 0.0, Jaro-Winkler Similarity: 0.5298460918819893
File: 05_transcription.txt, Cosine Similarity: 0.0, Jaro-Winkler Similarity: 0.5101395251615599
File: 06_transcription.txt, Cosine Similarity: 0.0, Jaro-Winkler Similarity: 0.5100472828169275
File: 07_transcription.txt, Cosine Similarity: 0.0, Jaro-Winkler Similarity: 0.5494827978600301
File: 09_transcription.txt, Cosine Similarity: 0.0, Jaro-Winkler Similarity: 0.5102529798669606
File: 11_transcription.txt, Cosine Similarity: 0.0, Jaro-Winkler Similarity: 0.510345047109753
File: 29_transcription.txt, Cosine Similarity: 0.4908236132007333, Jaro-Winkler Similarity: 0.510253764123733


In [17]:
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Define the directory containing the transcribed text files
transcribed_directory = r'D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\Output'

# Reference text for similarity calculation
reference_text = "linear regression"

# Function to calculate cosine similarity
def calculate_cosine_similarity(transcribed_text, reference_text):
    vectorizer = CountVectorizer().fit([transcribed_text, reference_text])
    vectorized_text = vectorizer.transform([transcribed_text, reference_text])
    cosine_sim = cosine_similarity(vectorized_text)[0][1]
    return cosine_sim

# List to store similarity scores along with filenames
similarity_scores = []

# Iterate through each text file in the directory
for filename in os.listdir(transcribed_directory):
    if filename.endswith('.txt'):
        file_path = os.path.join(transcribed_directory, filename)
        
        # Read the content of the text file
        with open(file_path, 'r') as file:
            transcribed_text = file.read()

        # Calculate cosine similarity
        cosine_sim = calculate_cosine_similarity(transcribed_text, reference_text)

        # Append the filename and cosine similarity score to the list
        similarity_scores.append((filename, cosine_sim))

# Sort the list of similarity scores based on cosine similarity
similarity_scores.sort(key=lambda x: x[1], reverse=True)

# Print the sorted results based on cosine similarity
print("Similarity Analysis based on Cosine Similarity:")
for filename, cosine_sim in similarity_scores:
    print(f"File: {filename}, Cosine Similarity: {cosine_sim}")

# Original video directory
video_directory = r'D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\Video'
print('----------------------------------------------------------------------------------------------')
# Iterate through the sorted similarity scores
for filename, cosine_sim in similarity_scores:
    # Exclude files with cosine similarity of 0
    if cosine_sim > 0:
        # Construct the filenames for video, audio, and transcribed text
        video_filename = os.path.join(video_directory, os.path.splitext(filename)[0] + '.mp4')

        # Print or use the filenames as needed
        print("Video File:", video_filename)


Similarity Analysis based on Cosine Similarity:
File: 29_transcription.txt, Cosine Similarity: 0.4908236132007333
File: 03_transcription.txt, Cosine Similarity: 0.004343804386977241
File: 01_transcription.txt, Cosine Similarity: 0.0
File: 02_transcription.txt, Cosine Similarity: 0.0
File: 04_transcription.txt, Cosine Similarity: 0.0
File: 05_transcription.txt, Cosine Similarity: 0.0
File: 06_transcription.txt, Cosine Similarity: 0.0
File: 07_transcription.txt, Cosine Similarity: 0.0
File: 09_transcription.txt, Cosine Similarity: 0.0
File: 11_transcription.txt, Cosine Similarity: 0.0
----------------------------------------------------------------------------------------------
Video File: D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\Video\29_transcription.mp4
Video File: D:\SRM\3rd_Year\6th_Semester\Research\Video_Retriveal_System\Data\Video\03_transcription.mp4


In [ ]:
 # File: 01.wav, Cosine Similarity: 0.005799075005326842, Jaro-Winkler Similarity: 0.5189410239758807 
# File: 02.wav, Cosine Similarity: 0.1521350346886033, Jaro-Winkler Similarity: 0.5187984998530961
# File: 03.wav, Cosine Similarity: 0.3262965315770913, Jaro-Winkler Similarity: 0.5373704074115231
# File: 04.wav, Cosine Similarity: 0.10446634907830515, Jaro-Winkler Similarity: 0.6299975302964496
# File: 05.wav, Cosine Similarity: 0.2771044236587905, Jaro-Winkler Similarity: 0.5003553449807521
# File: 06.wav, Cosine Similarity: 0.09556226218669801, Jaro-Winkler Similarity: 0.5372947136528841
# File: 07.wav, Cosine Similarity: 0.051082332851447504, Jaro-Winkler Similarity: 0.5375274735255118
# File: 09.wav, Cosine Similarity: 0.07073958731981543, Jaro-Winkler Similarity: 0.5560310290479082
# Error processing chunk: Calculated padded input size per channel: (2). Kernel size: (3). Kernel size can't be greater than actual input size
# File: 11.wav, Cosine Similarity: 0.0, Jaro-Winkler Similarity: 0.5190914749738279
# File: 29.wav, Cosine Similarity: 0.004828383156831224, Jaro-Winkler Similarity: 0.5375133409189137